# [서울시 따릉이 자전거 이용 예측 AI 모델 ]

## EDA 및 전처리

In [3]:
import pandas as pd

In [ ]:
# 데이터 다운로드 링크로 데이터를 코랩으로 가져옴

!wget 'https://bit.ly/3gLj0Q6'

import zipfile
with zipfile.ZipFile('3gLj0Q6', 'r') as existing_zip:
    existing_zip.extractall('data')

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [66]:
train.info() # 학습 데이터 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [ ]:
test.info() # 테스트 데이터 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


In [17]:
train.isnull().sum() # 학습 데이터 결측치 확인

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [ ]:
test.isnull().sum() # 테스트 데이터 결측치 확인

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [18]:
train = train.dropna() # 결측치 삭제
test = test.fillna(0) # 결측치 0으로 처리

## scikit-learn의 의사결정나무 모델 사용

In [6]:
import sklearn
from sklearn.tree import DecisionTreeRegressor

- 의사결정나무란 의사 결정 규칙과 그 결과들을 트리 구조로 도식화한 의사 결정 지원 도구의 일종으로, 스무고개 방식으로 구조화되는 것이라고 볼 수 있다. 각 행들은 피쳐들을 가지고 있는데, 이 중 하나의 피쳐를 정해서 해당 피쳐의 값에 대해 특정한 하나의 값을 정한다면, 이를 기준으로 모든 행들을 두 개의 노드로 분류할 수 있다. 

- 모델을 선언한 후 fit 함수를 사용해 모델을 훈련시킬 수 있는데, 이때 X 데이터는 예측에 사용되는 변수들이고, Y 데이터는 예측 결과 변수여야 한다.

In [14]:
x_train = train.drop(['count'],axis = 1)
y_train = train['count']

In [15]:
model = DecisionTreeRegressor()
model.fit(x_train,y_train)

DecisionTreeRegressor()

In [19]:
pred = model.predict(test)
pred[:5]

array([117., 173.,  38.,  18.,  93.])

## 제출용 파일에 예측 모델 결과 할당하기

In [28]:
submission = pd.read_csv('data/submission.csv')

In [30]:
submission['count'] = pred

In [31]:
submission

,id,count
0,0,117.0
1,1,164.0
2,2,114.0
3,4,49.0
4,5,92.0
...,...,...
710,2148,137.0
711,2149,137.0
712,2165,132.0
713,2166,213.0


## 모델 성능 개선(결측치를 개별 컬럼의 평균으로)

In [7]:
a = train

In [8]:
print(a.isnull().sum()) # 결측치 확인

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64


In [9]:
# 결측치를 개별 컬럼의 평균으로 대체
a.fillna({'hour_bef_temperature':int(a['hour_bef_temperature'].mean())}, inplace=True)
a.fillna({'hour_bef_precipitation':int(a['hour_bef_precipitation'].mean())}, inplace=True)
a.fillna({'hour_bef_windspeed':int(a['hour_bef_windspeed'].mean())}, inplace=True)
a.fillna({'hour_bef_humidity':int(a['hour_bef_humidity'].mean())}, inplace=True)
a.fillna({'hour_bef_visibility':int(a['hour_bef_visibility'].mean())}, inplace=True)
a.fillna({'hour_bef_ozone':int(a['hour_bef_ozone'].mean())}, inplace=True)
a.fillna({'hour_bef_pm10':int(a['hour_bef_pm10'].mean())}, inplace=True)
a.fillna({'hour_bef_pm2.5':int(a['hour_bef_pm2.5'].mean())}, inplace=True)

In [10]:
print(a.isnull().sum()) # 다시 확인

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64


# 모델 성능 개선(보간법 사용)

In [11]:
b = train

In [12]:
b.interpolate(inplace=True)
b.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

# 랜덤 포레스트 모델 사용

In [84]:
!pip install -U scikit-learn # 라이브러리 오류 발생으로 다시 설치 시도

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 1.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.10 requires scikit-learn==0.23.2, but you have scikit-learn 1.0.2 which is incompatible.


In [1]:
from sklearn.ensemble import RandomForestRegressor # 오류 해결 완료

In [27]:
a_train = train.drop(['count'], axis=1)
a_train_c = train['count']

In [28]:
model = RandomForestRegressor(criterion = 'mse') # 대회의 평가척도는 rmse
model.fit(a_train, a_train_c)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


RandomForestRegressor(criterion='mse')

In [29]:
model.feature_importances_ # 랜덤포레스트모델 예측변수의 중요도를 출력

array([0.02662649, 0.58770625, 0.18081542, 0.0197157 , 0.02619769,
       0.03933568, 0.03030302, 0.0381093 , 0.03078413, 0.02040632])

In [24]:
a_train.columns

Index(['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'],
      dtype='object')

In [32]:
b_train = train.drop(['count'], axis=1)
b_train_c = train['count']
model = RandomForestRegressor(criterion = 'mse') # 대회의 평가척도는 rmse
model.fit(b_train, b_train_c)
model.feature_importances_

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


array([0.02451848, 0.58989773, 0.18087009, 0.01908009, 0.02641055,
       0.03373663, 0.03252914, 0.04068144, 0.03081381, 0.02146206])

- 다수의 변수 중요도가 낮게 산출됨, 결측치를 보간법으로 채운 b 데이터셋도 동일한 방법으로 중요도 산출

In [33]:
# a에서 drop 할 피쳐의 경우에 수 대로 3개의 a_train을 생성

a_train_1 = train.drop(['count','id'], axis=1)
a_train_2 = train.drop(['count', 'id', 'hour_bef_windspeed'], axis=1)
a_train_3 = train.drop(['count', 'id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

# 각 train 에 따라 동일하게 피쳐를 drop 한 test 셋들을 생성

test_1 = test.drop(['id'], axis=1)
test_2 = test.drop(['id', 'hour_bef_windspeed'], axis=1)
test_3 = test.drop(['id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

- 관련도가 낮은 컬럼들 제외 후 모델 훈련

In [37]:
model_input_var1 = RandomForestRegressor(criterion = 'mse')
model_input_var1.fit(a_train_1, a_train_c)

model_input_var2 = RandomForestRegressor(criterion = 'mse')
model_input_var2.fit(a_train_2, a_train_c)

model_input_var3 = RandomForestRegressor(criterion = 'mse')
model_input_var3.fit(a_train_3, a_train_c)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


RandomForestRegressor(criterion='mse')

In [45]:
a_pred_1 = model_input_var1.predict(test_1)
a_pred_2 = model_input_var2.predict(test_2)
a_pred_3 = model_input_var3.predict(test_3)

In [49]:
a_submission_1 = pd.read_csv('data/submission.csv')
a_submission_2 = pd.read_csv('data/submission.csv')
a_submission_3 = pd.read_csv('data/submission.csv')

a_submission_1['count'] = a_pred_1
a_submission_2['count'] = a_pred_2
a_submission_3['count'] = a_pred_3

In [50]:
a_submission_1

,id,count
0,0,88.18
1,1,220.85
2,2,73.18
3,4,31.84
4,5,66.39
...,...,...
710,2148,61.35
711,2149,73.47
712,2165,122.75
713,2166,152.73


In [51]:
a_submission_2

,id,count
0,0,98.68
1,1,213.32
2,2,77.29
3,4,29.63
4,5,83.49
...,...,...
710,2148,58.44
711,2149,75.58
712,2165,122.20
713,2166,157.58


In [52]:
a_submission_3

,id,count
0,0,93.49
1,1,200.67
2,2,81.95
3,4,27.76
4,5,63.58
...,...,...
710,2148,61.11
711,2149,71.97
712,2165,125.83
713,2166,158.22


In [56]:
# b에서도 동일하게 3개의 b_train을 생성

b_train_1 = train.drop(['count','id'], axis=1)
b_train_2 = train.drop(['count', 'id', 'hour_bef_precipitation'], axis=1)
b_train_3 = train.drop(['count', 'id', 'hour_bef_precipitation', 'hour_bef_pm10'], axis=1)

In [57]:
model_input_var4 = RandomForestRegressor(criterion = 'mse')
model_input_var4.fit(b_train_1, b_train_c)

model_input_var5 = RandomForestRegressor(criterion = 'mse')
model_input_var5.fit(b_train_2, b_train_c)

model_input_var6 = RandomForestRegressor(criterion = 'mse')
model_input_var6.fit(b_train_3, b_train_c)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


RandomForestRegressor(criterion='mse')

In [58]:
b_pred_1 = model_input_var1.predict(test_1)
b_pred_2 = model_input_var2.predict(test_2)
b_pred_3 = model_input_var3.predict(test_3)

In [59]:
b_submission_1 = pd.read_csv('data/submission.csv')
b_submission_2 = pd.read_csv('data/submission.csv')
b_submission_3 = pd.read_csv('data/submission.csv')

b_submission_1['count'] = a_pred_1
b_submission_2['count'] = a_pred_2
b_submission_3['count'] = a_pred_3

In [60]:
b_submission_1

,id,count
0,0,88.18
1,1,220.85
2,2,73.18
3,4,31.84
4,5,66.39
...,...,...
710,2148,61.35
711,2149,73.47
712,2165,122.75
713,2166,152.73


In [61]:
b_submission_2

,id,count
0,0,98.68
1,1,213.32
2,2,77.29
3,4,29.63
4,5,83.49
...,...,...
710,2148,58.44
711,2149,75.58
712,2165,122.20
713,2166,157.58


In [62]:
b_submission_3

,id,count
0,0,93.49
1,1,200.67
2,2,81.95
3,4,27.76
4,5,63.58
...,...,...
710,2148,61.11
711,2149,71.97
712,2165,125.83
713,2166,158.22


- a와 b의 결과가 동일하게 나오는 것을 확인할 수 있었습니다.

## 그렇다면, 중요도가 특정 수치 이하인 컬럼을 모두 제거하고 돌려도 동일한 결과가 나올까?

In [63]:
b_train = train.drop(['count'], axis=1)
b_train_c = train['count']
model = RandomForestRegressor(criterion = 'mse') # 대회의 평가척도는 rmse
model.fit(b_train, b_train_c)
model.feature_importances_ 

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


array([0.02516954, 0.58525538, 0.18314518, 0.01802785, 0.02635487,
       0.0365592 , 0.03185634, 0.04093335, 0.03239439, 0.0203039 ])

In [64]:
b_train.columns

Index(['id', 'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'],
      dtype='object')

- 중요도가 0.1 미만인 컬럼을 모두 삭제한 후 위의 과정을 반복해 보겠습니다.

In [75]:
b_train_4 = train.drop(['count','id', 'hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'], axis = 1)
test_4 = test.drop(['id', 'hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'], axis = 1)

In [76]:
model_input_var7 = RandomForestRegressor(criterion = 'mse')
model_input_var7.fit(b_train_4, b_train_c)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


RandomForestRegressor(criterion='mse')

In [77]:
b_pred_4 = model_input_var7.predict(test_4)
b_submission_4 = pd.read_csv('data/submission.csv')
b_submission_4['count'] = b_pred_4
b_submission_4

,id,count
0,0,107.82
1,1,258.96
2,2,113.27
3,4,36.00
4,5,83.29
...,...,...
710,2148,61.25
711,2149,70.00
712,2165,122.09
713,2166,194.71


- 결과가 이전과는 다름을 확인할 수 있습니다.